먼저, 주피터 노트북에서 웹 크롤링을 위한 파이썬 라이브러리인 BeautifulSoup, requests, 그리고 Selenium을 설치해야 합니다.

In [1]:
%pip install beautifulsoup4 requests selenium lxml openpyxl

Note: you may need to restart the kernel to use updated packages.


그런 다음, 아래의 코드를 실행하여 웹드라이버를 설정합니다. 웹드라이버는 웹사이트를 자동으로 탐색하기 위해 사용됩니다. 이 예에서는 Chrome 웹드라이버를 사용하겠습니다. 웹드라이버를 사용하려면 해당 브라우저의 웹드라이버를 설치해야 합니다.

In [2]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import pandas as pd

# 웹드라이버를 설정합니다. (여기서는 Chrome을 사용합니다.)
# 웹드라이버의 경로를 자신의 시스템에 맞게 설정해주세요.
options = Options()
options.page_load_strategy = 'eager'
driver = webdriver.Chrome(options=options)

# 웹사이트에 접속합니다.
driver.get('https://data.kleague.com')

# Find frameset element and get its children
frames = driver.find_elements("tag name", 'frame')

for frame in frames:
    if 'https://portal.kleague.com' in frame.get_attribute('src'):
        # redirect the browser to the frame url
        driver.get(frame.get_attribute('src'))
    else:
        # do nothing
        pass


onclick 속성에서 데이터를 추출하는 함수를 포함한 코드입니다. 사이트 구조상 onClick에서 데이터를 자주 추출해야함. 

In [3]:
import re

def extract_onclick_data(onclick):
    if "moveMainFrameMcPlayer" in onclick:
        match = re.search(r"moveMainFrameMcPlayer\('(.+?)','(.+?)','(.+?)'\)", onclick)
        if match:
            return dict(zip(['menuCd', 'playerId', 'teamId'], match.groups()))
    elif "moveMainFrame" in onclick:
        match = re.search(r"moveMainFrame\('(.+?)'\)", onclick)
        if match:
            return {'menuCd': match.group(1)}
    return {}


In [4]:
# JavaScript를 실행하여 페이지를 이동합니다.
# driver.execute_script("moveMainFrame('0011')")
driver.execute_script("moveMainFrame('0415')")

In [6]:
# 페이지의 HTML을 가져옵니다.
html = driver.page_source

# BeautifulSoup 객체를 생성합니다.
soup = BeautifulSoup(html, 'html.parser')


# 특정 form 안에 있는 모든 테이블을 찾아 데이터를 스크랩핑합니다.
form = soup.find('form', {'id': 'frm'})
table_data = []
if form is not None:
    tables = form.find_all('table')
    for table in tables:
        # Get column names from thead
        column_names = [col.text for col in table.thead.find_all('th')]
        # Get all rows in the table
        rows = table.find_all('tr')
        row_data = []
        for row in rows:
            # Get all columns in the row
            cols = row.find_all('td')
            # Get onclick attribute value
            onclick = row.get('onclick')
            # Append the onclick value to the row data if it exists
            cols_data = dict(zip(column_names, [col.text for col in cols]))
            if onclick is not None:
                cols_data.update(extract_onclick_data(onclick))
            row_data.append(cols_data)
        table_data.append(pd.DataFrame(row_data))

# 데이터를 출력합니다.
for data in table_data:
    print(data)

# Export data to excel
with pd.ExcelWriter('kl_data.xlsx') as writer:
    for i, data in enumerate(table_data):
        data.to_excel(writer, sheet_name=f'Sheet{i+1}')



      소속 클럽   등번   성명      키   몸무게        생년월일 menuCd  playerId teamId
0       NaN  NaN  NaN    NaN   NaN         NaN    NaN       NaN    NaN
1      강원FC   25  김정호  184Cm  82Kg  1998/04/07   0416  20170124    K21
2      강원FC    1  유상훈  194Cm  84Kg  1989/05/25   0416  20110094    K21
3      강원FC   31  이광연  184Cm  85Kg  1999/09/11   0416  20190058    K21
4      강원FC   41  조민규  193Cm  87Kg  2003/04/30   0416  20230068    K21
..      ...  ...  ...    ...   ...         ...    ...       ...    ...
95  충북 청주FC   25  정현호  196Cm  85Kg  2004/11/26   0416  20230204    K37
96  포항 스틸러스    1  윤평국  189Cm  85Kg  1992/02/08   0416  20130151    K03
97  포항 스틸러스   32  이승환  187Cm  78Kg  2003/04/05   0416  20220160    K03
98  포항 스틸러스   41  조성훈  189Cm  85Kg  1998/04/21   0416  20190122    K03
99  포항 스틸러스   21  황인재  187Cm  73Kg  1994/04/22   0416  20160079    K03

[100 rows x 9 columns]
       소속 클럽   등번   성명      키   몸무게        생년월일 menuCd  playerId teamId
0        NaN  NaN  NaN    NaN   NaN         NaN    N

선수 이름을 검색하여 해당 선수의 playerId와 teamId를 찾고, 이 정보를 사용하여 페이지를 이동시키는 코드를 추가

In [7]:
# Search for a specific player
player_name = "김주찬"  # replace with the name of the player you are looking for
for data in table_data:
    player_data = data[data['성명'] == player_name]
    if not player_data.empty:
        player_id = player_data['playerId'].values[0]
        team_id = player_data['teamId'].values[0]
        # Move to the page with the player's details
        driver.execute_script(f"moveMainFrameMcPlayer('0416', '{player_id}', '{team_id}')")
        break

In [8]:
import pandas as pd

# Execute JavaScript to get the value of resultDataSet
result_data_set = driver.execute_script("return resultDataSet;")

# Check if resultDataSet is a list
if isinstance(result_data_set, list):
    print(f"resultDataSet contains {len(result_data_set)} objects.")
    for i, obj in enumerate(result_data_set):
        # Convert the object to a DataFrame
        df = pd.DataFrame(obj)
        # Print the DataFrame
        print(f"Object {i+1}:")
        print(df)
else:
    print(f"resultDataSet is not a list. It is a {type(result_data_set)}.")

# Export data to excel with name of player_name as sheet name
with pd.ExcelWriter('kl_player_data.xlsx') as writer:
    for i, obj in enumerate(result_data_set):
        # Convert the object to a DataFrame
        df = pd.DataFrame(obj)
        df.to_excel(writer, sheet_name=f"{player_name}_{i+1}", index=False)

resultDataSet contains 7 objects.
Object 1:
   Team_S_Name Team_id
0           강원     K21
1           광주     K22
2           대구     K17
3           대전     K10
4           서울     K09
5           수원     K02
6         수원FC     K29
7           울산     K01
8           인천     K18
9           전북     K05
10          제주     K04
11          포항     K03
Object 2:
   PLAYER_ID   game_date  game_id game_time  meet_seq meet_year  round_id  \
0   20230103  2023/02/25      3.0     16:30         1      2023         1   
1   20230103  2023/03/05     10.0     14:00         1      2023         2   
2   20230103  2023/03/11     13.0     14:00         1      2023         3   
3   20230103  2023/03/19     23.0     14:00         1      2023         4   
4   20230103  2023/04/02     30.0     16:30         1      2023         5   
5   20230103  2023/04/08     31.0     14:00         1      2023         6   
6        NaN  2023/04/15     37.0     14:00         1      2023         7   
7   20230103  2023/04/22     43